<a href="https://colab.research.google.com/github/shashigharti/60daysofudacity-challenge/blob/master/Final_Project_Federated_Learning_with_Additive_Secret_Sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project**
Federated Learning with Trusted Aggregator ;
Aggregate gradients using additive secret sharing and fixed precision encoding

In [0]:
!pip install syft

In [153]:
hook = sy.TorchHook(th)


W0711 11:17:05.043093 139749766735744 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
import syft as sy
import torch as th
from torch import nn, optim
import torch.nn.functional as F

In [145]:
hook

In [0]:
#create three workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = th.tensor([[0],[0],[1],[1.]])

In [0]:
#initialize a toy model
# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        return x
model = Net()

#initialize a toy model
model = nn.Linear(2,1)

In [159]:
# Encode everything
data = data.fix_prec().share(bob, alice, crypto_provider=secure_worker, requires_grad=True)
target = target.fix_prec().share(bob, alice, crypto_provider=secure_worker, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=secure_worker, requires_grad=True)
print(data)


(Wrapper)>AutogradTensor>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:64826375887 -> bob:23175403454]
	-> (Wrapper)>[PointerTensor | me:58331535930 -> alice:4228534626]
	*crypto provider: secure_worker*


In [160]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) print our progress
    print(loss.get().float_precision())
   

tensor([3.3130])
tensor([0.6270])
tensor([0.2930])
tensor([0.1830])
tensor([0.1190])
tensor([0.0800])
tensor([0.0510])
tensor([0.0340])
tensor([0.0230])
tensor([0.0160])
tensor([0.0120])
tensor([0.0070])
tensor([0.0040])
tensor([0.0030])
tensor([0.0020])
tensor([0.0010])
tensor([0.])
tensor([0.0010])
tensor([0.0010])
tensor([0.])
